In [1]:
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols, Point, ReferenceFrame

# ∑ = Barre1
# géométrie
r_11, r_12, m_1, theta10, g = sp.symbols(
    'r_11 r_12 m_1 theta10 g', real=True, positive=True, nonzero=True)
# paramètres de position
theta1 = sp.symbols('theta1', real=True)
# efforts
X01, Y01, F_1, T = sp.symbols('X_01 Y_01 F_1 T')
# réologie : facteur d'amortisement et raideur du ressort
k_1 = sp.symbols('k_1', real=True, positive=True, nonzero=True)

# Repères attachées au pièces en utilisant des RéférenceFrame plus adaptés à la mécanique des systèmes qu'un CoordSys3D
R0 = ReferenceFrame('R_0')
R1 = ReferenceFrame('R_1')

# positionnement des repères
# R1 se déduit de R0 par rotation d'angle theta1 autour de z0=z1
R1.orient(R0, 'Axis', [theta1, R0.z])

# Points caractéristiques
O_1 = Point('O_1')  # origine
A_1 = Point('A_1')
B_1 = Point('B_1')
G_1 = Point('G_1')

# Et position associées
O_1.set_vel(R0, 0)  # O est un point de R0
O_1.set_vel(R1, 0)  # O est un point de R01

A_1.set_pos(O_1, -r_11 * R1.x)
A_1.set_vel(R1, 0)  # A est un point de R1

B_1.set_pos(O_1, r_12 * R1.x)  # position dans R0
B_1.set_vel(R1, 0)  # B est un point de R1

G_1.set_vel(R1, 0)
G_1.set_pos(O_1, 0*R1.x)

# Torseurs
R_01 = X01 * R0.x + Y01 * R0.y
M_01 = 0 * R0.z

R_P1 = -m_1*g*R1.y  # hypothèse la tige est à l'horizontale
M_P1 = G_1.pos_from(O_1).cross(R_P1)

R_Fr1 = 0*R0.x
M_Fr1 = k_1*(theta1-theta10)*R0.z

R_T = T*R1.y  # hypothèse la tension est perpendiculaire à la tige
M_T = B_1.pos_from(O_1).cross(R_T)

R_F1 = F_1*R1.y
M_F1 = A_1.pos_from(O_1).cross(R_F1)

# Théorème de la résultante statique
R_x = R_01.dot(R0.x) + R_P1.dot(R0.x) + R_Fr1.dot(R0.x) + \
    R_T.dot(R0.x) + R_F1.dot(R0.x)
R_y = R_01.dot(R0.y) + R_P1.dot(R0.y) + R_Fr1.dot(R0.y) + \
    R_T.dot(R0.y) + R_F1.dot(R0.y)
Eq_Rx_1 = sp.Eq(R_x, 0)
Eq_Ry_1 = sp.Eq(R_y, 0)
print('Théorème de la résultante ∑ = Barre1')
# print(Eq_Rx_1, Eq_Ry_1)
display(Eq_Rx_1, Eq_Ry_1)

# Théorème du moment en O_1
M_O_1_z = M_01.dot(R0.z) + M_P1.dot(R0.z) + \
    M_Fr1.dot(R0.z) + M_T.dot(R0.z) + M_F1.dot(R0.z)
Eq_Mz_O_1 = sp.Eq(M_O_1_z, 0)
print('Théorème du moment, en O_1')
display(Eq_Mz_O_1)


# ∑ = Barre2
# géométrie
r_21, r_22, m_2, theta20, g = sp.symbols(
    'r_21 r_22 m_1 theta20 g', real=True, positive=True, nonzero=True)
# paramètres de position
theta2 = sp.symbols('theta2', real=True)
# efforts
X01p, Y01p, F_2, Tp = sp.symbols("X'_01 Y'_01 F_2 T'")
# réologie : facteur d'amortisement et raideur du ressort
k_2 = sp.symbols('k_2', real=True, positive=True, nonzero=True)

# Repères attachées au pièces en utilisant des RéférenceFrame plus adaptés à la mécanique des systèmes qu'un CoordSys3D
R2 = ReferenceFrame('R_2')

# positionnement des repères
# R2 se déduit de R0 par rotation d'angle theta2 autour de z0=z1
R2.orient(R0, 'Axis', [theta2, R0.z])

# Points caractéristiques
O_2 = Point('O_2')  # origine
A_2 = Point('A_2')
B_2 = Point('B_2')
G_2 = Point('G_2')

# Et position associées
O_2.set_vel(R0, 0)  # O est un point de R0
O_2.set_vel(R2, 0)  # O est un point de R02

A_2.set_pos(O_2, -r_21 * R2.x)
A_2.set_vel(R2, 0)  # A est un point de R2

B_2.set_pos(O_2, r_22 * R2.x)  # position dans R0
B_2.set_vel(R2, 0)  # B est un point de R2

G_2.set_vel(R2, 0)
G_2.set_pos(O_2, 0*R2.x)

# Torseurs
R_01p = X01p * R0.x + Y01p * R0.y
M_01p = 0 * R0.z

R_P2 = -m_2*g*R2.y  # hypothèse la tige est à l'horizontale
M_P2 = G_2.pos_from(O_2).cross(R_P2)

R_Fr2 = 0*R0.x
M_Fr2 = k_2*(theta2-theta20)*R0.z

R_Tp = Tp*R2.y  # hypothèse la tension est perpendiculaire à la tige
M_Tp = B_2.pos_from(O_2).cross(R_Tp)

R_F2 = F_2*R2.y
M_F2 = A_2.pos_from(O_2).cross(R_F2)

# Théorème de la résultante statique
R_xp = R_01p.dot(R0.x) + R_P2.dot(R0.x) + R_Fr2.dot(R0.x) + \
    R_Tp.dot(R0.x) + R_F2.dot(R0.x)
R_yp = R_01p.dot(R0.y) + R_P2.dot(R0.y) + R_Fr2.dot(R0.y) + \
    R_Tp.dot(R0.y) + R_F2.dot(R0.y)
Eq_Rx_2 = sp.Eq(R_xp, 0)
Eq_Ry_2 = sp.Eq(R_yp, 0)
print('Théorème de la résultante ∑ = Barre2')
# print(Eq_Rx_2, Eq_Ry_2)
display(Eq_Rx_2, Eq_Ry_2)

# Théorème du moment en O_2
M_O_2_z = M_01p.dot(R0.z) + M_P2.dot(R0.z) + \
    M_Fr2.dot(R0.z) + M_Tp.dot(R0.z) + M_F2.dot(R0.z)
Eq_Mz_O_2 = sp.Eq(M_O_2_z, 0)
print('Théorème du moment, en O_2')
display(Eq_Mz_O_2)


Théorème de la résultante ∑ = Barre1


Eq(-F_1*sin(theta1) - T*sin(theta1) + X_01 + g*m_1*sin(theta1), 0)

Eq(F_1*cos(theta1) + T*cos(theta1) + Y_01 - g*m_1*cos(theta1), 0)

Théorème du moment, en O_1


Eq(-F_1*r_11 + T*r_12 + k_1*(theta1 - theta10), 0)

Théorème de la résultante ∑ = Barre2


Eq(-F_2*sin(theta2) - T'*sin(theta2) + X'_01 + g*m_1*sin(theta2), 0)

Eq(F_2*cos(theta2) + T'*cos(theta2) + Y'_01 - g*m_1*cos(theta2), 0)

Théorème du moment, en O_2


Eq(-F_2*r_21 + T'*r_22 + k_2*(theta2 - theta20), 0)